In [5]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.version)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.10/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [87]:
from google.colab import files

training_data = files.upload()

Saving training_data.csv to training_data.csv


In [88]:
import pandas as pd

game_data = pd.read_csv("training_data.csv")

In [89]:
game_data.head()

,PPG_x,FGP2_x,FGP3_x,P3FGA_x,FTP_x,FTAPG_x,ORR_x,DRR_x,APG_x,TOPG_x,...,OFGP2_y,OFGP3_y,OP3FGA_y,OFTAPG_y,TOFPG_y,WP_y,SOS_y,WPAFH_y,CGWP_y,Win
0,71.206897,0.461333,0.360153,0.316940,0.762768,20.931034,0.328413,0.572402,13.034483,16.206897,...,0.487019,0.325180,0.383554,18.733333,14.333333,0.600000,0.421069,0.437500,0.647059,1
1,85.214286,0.510172,0.350534,0.305435,0.701429,25.000000,0.354167,0.616733,17.642857,14.785714,...,0.466667,0.329582,0.389480,15.758621,13.000000,0.655172,0.451787,0.526316,0.583333,1
2,75.965517,0.523735,0.317808,0.221212,0.669737,26.206897,0.357014,0.625926,15.551724,14.000000,...,0.439407,0.322034,0.316448,20.724138,15.068966,0.793103,0.522585,0.692308,0.714286,1
3,79.344828,0.569444,0.380769,0.340314,0.765753,25.172414,0.331176,0.613079,15.620690,18.241379,...,0.463072,0.341102,0.258488,16.696970,17.060606,0.878788,0.498915,0.750000,0.777778,1
4,74.482759,0.502068,0.376518,0.290076,0.685512,19.517241,0.299632,0.625664,16.000000,14.172414,...,0.489852,0.337255,0.319950,21.200000,14.633333,0.600000,0.548989,0.357143,0.555556,1


In [90]:
X = game_data.iloc[:, :-1]  # Features (all columns except the last one)
y = game_data.iloc[:, -1]   # Target variable (last column)

In [116]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=9)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [123]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout

model = Sequential()
model.add(Dense(80, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)))
model.add(Dropout(0.4))
model.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))

In [124]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [126]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=40, epochs=12)

Epoch 1/12
56/56 [==============================] - 1s 5ms/step - loss: 0.6016 - accuracy: 0.7150 - val_loss: 0.6013 - val_accuracy: 0.7089
Epoch 2/12
56/56 [==============================] - 0s 4ms/step - loss: 0.5984 - accuracy: 0.7128 - val_loss: 0.5987 - val_accuracy: 0.7241
Epoch 3/12
56/56 [==============================] - 0s 4ms/step - loss: 0.5927 - accuracy: 0.7195 - val_loss: 0.5967 - val_accuracy: 0.7038
Epoch 4/12
56/56 [==============================] - 2s 44ms/step - loss: 0.5904 - accuracy: 0.7262 - val_loss: 0.5953 - val_accuracy: 0.6962
Epoch 5/12
56/56 [==============================] - 0s 3ms/step - loss: 0.5844 - accuracy: 0.7289 - val_loss: 0.5924 - val_accuracy: 0.6987
Epoch 6/12
56/56 [==============================] - 0s 4ms/step - loss: 0.5872 - accuracy: 0.7217 - val_loss: 0.5874 - val_accuracy: 0.7038
Epoch 7/12
56/56 [==============================] - 0s 4ms/step - loss: 0.5874 - accuracy: 0.7177 - val_loss: 0.5870 - val_accuracy: 0.6886
Epoch 8/12
56/56 [=

In [127]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)

13/13 [==============================] - 0s 5ms/step - loss: 0.5905 - accuracy: 0.7038


In [166]:
data2024 = files.upload()

Saving matchup_data2024.csv to matchup_data2024.csv


In [171]:
matchups2024 = pd.read_csv("matchup_data2024.csv")

X_2024 = matchups2024.iloc[:, 4:]  # Features (all columns except the last one)
games_2024 = matchups2024.iloc[:, :4]   # Target variable (last column)

X_2024 = scaler.transform(X_2024)

In [172]:
games_2024['probs'] = model.predict(X_2024)

143/143 [==============================] - 1s 5ms/step


In [173]:
games_2024

,Team X,Team Y,Name_x,Name_y,probs
0,1163,1235,Uconn,Iowa St.,0.552265
1,1163,1228,Uconn,Illinois,0.788964
2,1163,1120,Uconn,Auburn,0.630390
3,1163,1361,Uconn,SDSU,0.672422
4,1163,1140,Uconn,BYU,0.735193
...,...,...,...,...,...
4551,1160,1443,Colorado,W. Kentucky,0.790350
4552,1160,1255,Colorado,Longwood,0.802182
4553,1160,1438,Colorado,UVA,0.551349
4554,1160,1286,Colorado,Montanta St.,0.814845


In [189]:
two_probs = pd.merge(games_2024, games_2024,
                     left_on = ['Team Y', 'Team X'], right_on=['Team X', 'Team Y'],
                     how='inner')


two_probs['sum'] = two_probs['probs_x'] + two_probs['probs_y']
two_probs['average'] = (two_probs['probs_x'] / two_probs['sum'])
two_probs['Odds'] = two_probs['average'].apply(lambda x: (100/x)-100 if x < 0.5 == 0 else 100*x/(x-1))

two_probs = two_probs[['Team X_x', 'Team Y_x', 'Name_x_x', 'Name_y_x', 'average',
           'Odds']].rename(columns = {'Team X_x' : 'TeamID1',
                                      'Team Y_x' : 'TeamID2',
                                      'Name_x_x' : 'Name1',
                                      'Name_y_x' : 'Name2',
                                      'average' : 'Win_prob1',
                                      'Odds' : 'Odds1'})

two_probs.to_csv('probabilities2024.csv', index=False)